In [10]:
"""
-*- coding: utf-8 -*-
@datetime: 2021-06-28
@author  : jiangmingchao@joyy.sg
@describe: Imagenet dataset
"""
from math import e
import torch
import random 
import numpy as np 
import urllib.request as urt 

from PIL import Image 
from io import BytesIO
from torch.utils.data.dataset import Dataset
from torchvision.transforms import transforms as imagenet_transforms
from timm.data.auto_augment import rand_augment_transform, augment_and_mix_transform, auto_augment_transform
from timm.data.transforms import _pil_interp


class ImageDataset(Dataset):
    def __init__(self,
                image_file,
                train_phase,
                input_size,
                crop_size,
                shuffle = True ,
                interpolation='random',
                auto_augment = None, 
                color_prob = 0.4,
                hflip_prob = 0.5,
                vflip_prob = 0.0,

                ) -> None:
        super(ImageDataset, self).__init__()
        self.image_file = image_file
        self.image_list = [x.strip() for x in open(self.image_file).readlines()]
        self.length = [x for x in range(len(self.image_list))]
        self.train_phase = train_phase
        self.input_size = input_size
        self.crop_size = crop_size
        self.shuffle = shuffle
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]
        self.hflip_prob = hflip_prob
        self.vflip_prob = vflip_prob
        
        if self.shuffle and self.train_phase:
            for _ in range(10):
                random.shuffle(self.image_list)

        self.colorjitter_prob = None if color_prob is None else (color_prob, )*3 
        self.auto_augment = auto_augment
        self.interpolation = interpolation

        # train
        if self.train_phase:
            basic_tf = [
                imagenet_transforms.RandomResizedCrop((self.crop_size, self.crop_size)),
                imagenet_transforms.RandomHorizontalFlip(self.hflip_prob),
                imagenet_transforms.RandomVerticalFlip(self.vflip_prob),
            ]

            auto_tf = []
            if self.auto_augment:
                assert isinstance(auto_augment, str)
                if isinstance(self.crop_size, (tuple, list)):
                    img_size_min = min(self.crop_size)
                else:
                    img_size_min = self.crop_size 
                
                aa_params = dict(
                    translate_dict = int(img_size_min * 0.45),
                    img_mean = tuple([min(255, round(255 * x)) for x in self.mean])
                )
                if self.interpolation and self.interpolation != "random":
                    aa_params['interpolation'] = _pil_interp(self.interpolation)
                # rand aug
                if auto_augment.startswith('rand'):
                    auto_tf += [rand_augment_transform(auto_augment, aa_params)]
                # augmix
                elif auto_augment.startswith('augmix'):
                    aa_params['translate_pct'] = 0.3
                    auto_tf += [augment_and_mix_transform(auto_augment, aa_params)]
                # auto aug
                else:
                    auto_tf += [auto_augment_transform(auto_augment, aa_params)]

            if self.colorjitter_prob is not None:
                auto_tf += [
                    imagenet_transforms.ColorJitter(*self.colorjitter_prob)
                ]

            final_tf = [
                imagenet_transforms.ToTensor(),
                imagenet_transforms.Normalize(
                    mean= self.mean,
                    std= self.std 
                )
            ]
            self.data_aug = imagenet_transforms.Compose(
                basic_tf + auto_tf 
            )
            
            print(self.data_aug)

        # test 
        else:
            self.data_aug = imagenet_transforms.Compose([
                imagenet_transforms.Resize(self.input_size),
                imagenet_transforms.CenterCrop((self.crop_size, self.crop_size)),
                imagenet_transforms.ToTensor(),
                imagenet_transforms.Normalize(
                    mean = self.mean,
                    std = self.std 
                )
            ])
            
    def _decode_image(self, image_path):
        if "http" in image_path:
            image = Image.open(BytesIO(urt.urlopen(image_path).read()))
        else:
            image = Image.open(image_path)
        
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        return image


    def __getitem__(self, index):
        for _ in range(10):
            try:
                line = self.image_list[index]
                image_path, image_label = line.split(',')[0], line.split(',')[1]
                image = self._decode_image(image_path)
                image = self.data_aug(image)
                label = torch.from_numpy(np.array(int(image_label))).long()
                return image, label 
            except Exception as e:
                index = random.choice(self.length)
                print(f"The exception is {e}, image path is {image_path}!!!")
                

    def __len__(self):
        return len(self.image_list)



In [34]:
train_file = "/data/jiangmingchao/data/dataset/imagenet/val_oss_imagenet_128w.txt"
train_dataset = ImageDataset(
    image_file=train_file,
    train_phase=False,
    input_size = 224, 
    crop_size= 224,
    shuffle=False,
    interpolation='bilinear',
    auto_augment="rand",
    color_prob = None,
    hflip_prob = 0.0,
    vflip_prob = 0.0
    
)
print(train_dataset)
print(len(train_dataset))
for _ in range(10):
    for idx, data in enumerate(train_dataset):
#         plt.figure()
#         plt.imshow(data[0])
        print(data[0].shape)
        break

50000
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [35]:
import matplotlib.pyplot as plt
import torch.nn as nn 

In [36]:


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 3, 1, 1)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, 3, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(64)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        
        x = self.pool1(x)
        
        return x 

In [37]:
inputs = torch.randn(1, 3, 16, 16)

In [38]:
net = Net()

In [39]:
outputs = net(inputs)

In [40]:
outputs.shape

torch.Size([1, 64, 1, 1])

ModuleAttributeError: 'Net' object has no attribute 'module'